In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision import datasets
from torch.utils.tensorboard import SummaryWriter

# Define the transformation to normalize the images
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST dataset
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the model (e.g., a simple CNN)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

writer = SummaryWriter(log_dir='runs/mnist_experiment')

# Train the model
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            writer.add_scalar('training_loss', running_loss / 100, epoch * len(trainloader) + i)
            running_loss = 0.0

print("Finished Training")

# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")


[1, 100] loss: 2.266
[1, 200] loss: 2.103
[1, 300] loss: 1.517
[1, 400] loss: 0.762
[1, 500] loss: 0.484
[1, 600] loss: 0.374
[1, 700] loss: 0.324
[1, 800] loss: 0.280
[1, 900] loss: 0.259
[2, 100] loss: 0.233
[2, 200] loss: 0.235
[2, 300] loss: 0.219
[2, 400] loss: 0.191
[2, 500] loss: 0.179
[2, 600] loss: 0.166
[2, 700] loss: 0.158
[2, 800] loss: 0.163
[2, 900] loss: 0.155
[3, 100] loss: 0.140
[3, 200] loss: 0.130
[3, 300] loss: 0.130
[3, 400] loss: 0.128
[3, 500] loss: 0.121
[3, 600] loss: 0.121
[3, 700] loss: 0.122
[3, 800] loss: 0.116
[3, 900] loss: 0.106


In [4]:
from torchsummary import summary
print(summary(model,(1,28,28)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 14, 14]          18,496
            Linear-3                  [-1, 128]         401,536
            Linear-4                   [-1, 10]           1,290
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.29
Params size (MB): 1.61
Estimated Total Size (MB): 1.90
----------------------------------------------------------------
None
